In [3]:
import spacy
import pandas as pd
from afinn import Afinn
import numpy as np

nlp = spacy.load("en_core_web_md")

afinn_df = pd.read_csv("data/AFINN-en-165.tsv", sep="\t", header=None, names=["word", "score"])
#afinn_df
afinn = Afinn()

vad_df = pd.read_csv("data/NRC-VAD-Lexicon.tsv", sep="\t")
#vad_df

In [5]:
afinn_scores = dict(zip(afinn_df["word"], afinn_df["score"]))
#afinn_scores["yummy"]
def afinn_score(text):
    return sum(afinn_scores.get(t.lemma_, 0) for t in nlp(text))

word_valence = dict(zip(vad_df["Word"], vad_df["Valence"]))
word_arousal = dict(zip(vad_df["Word"], vad_df["Arousal"]))
word_dminanc = dict(zip(vad_df["Word"], vad_df["Dominance"]))

def text_valence(text, agg=sum):
    return agg([word_valence.get(t.lemma_, 0) for t in nlp(text)])

def text_arousal(text, agg=sum):
    return agg([word_arousal.get(t.lemma_, 0) for t in nlp(text)])

def text_dminanc(text, agg=sum):
    return agg([word_dminanc.get(t.lemma_, 0) for t in nlp(text)])

In [6]:
tweetDf = pd.read_csv("data/500000tweetshydrated.tsv", sep="\t")
tweetDf["full_text"] = tweetDf["full_text"].astype("string")
#tweetDf.dtypes
#print(tweetDf.full_text)
#tweetDf["wordList"] = wordList(tweetDf["full_text"].tolist())

In [7]:
def calc(row):
    return [row['Datetime'],
    afinn_score(row['full_text']),
    afinn.score(row['full_text']),
    text_valence(row['full_text']),
    text_arousal(row['full_text']),
    text_dminanc(row['full_text']),
    text_valence(row['full_text'], agg=np.mean),
    text_arousal(row['full_text'], agg=np.mean),
    text_dminanc(row['full_text'], agg=np.mean)]

from multiprocessing.dummy import Pool as ThreadPool
pool = ThreadPool(8)

counting = 0
score = []
for index, row in tweetDf.iterrows():
    #print(row['full_text'], afinn.score(row['full_text']))
    score = pool.map(calc, row)
    counting+=1
    if (counting % 1000 == 0): print(counting/len(tweetDf.index)*100, '%,', counting, '/', len(tweetDf.index))
    if counting >= 10: break

scoredf = pd.DataFrame(score, columns = ['date','AfinnSentiment','AfinnDefaultSentiment','NRCValence','NRCArousal','NRCValenceMean','NRCArousalMean'])

TypeError: 'int' object is not subscriptable

In [8]:
def calc(row):
    print("hey")
    return [row['Datetime'],
    afinn_score(row['full_text']),
    afinn.score(row['full_text']),
    text_valence(row['full_text']),
    text_arousal(row['full_text']),
    text_valence(row['full_text'], agg=np.mean),
    text_arousal(row['full_text'], agg=np.mean)]

from multiprocessing.dummy import Pool as ThreadPool
pool = ThreadPool(8)

counting = 0
score = []
#print(row['full_text'], afinn.score(row['full_text']))
score = pool.map(calc, tweetDf)

scoredf = pd.DataFrame(score, columns = ['date','AfinnSentiment','AfinnDefaultSentiment','NRCValence','NRCArousal','NRCValenceMean','NRCArousalMean'])

hey


In [71]:
scoredf.to_csv("data/500000TweetsHydratedScored.tsv",index = False,sep='\t',line_terminator='\r\n')

,date,AfinnSentiment,AfinnDefaultSentiment,NRCValence,NRCArousal,NRCValenceMean,NRCArousalMean
0,2020-04-06 17:22:41+00:00,2,2.0,4.739,4.000,0.225667,0.190476
1,2020-03-24 07:57:44+00:00,5,5.0,4.758,3.277,0.093294,0.064255
2,2019-10-20 06:20:39+00:00,-6,-4.0,12.256,12.685,0.211310,0.218707
3,2020-04-16 18:32:49+00:00,-3,-6.0,2.463,2.712,0.102625,0.113000
4,2020-03-09 00:03:08+00:00,0,0.0,7.596,5.430,0.180857,0.129286
...,...,...,...,...,...,...,...
95,2019-11-19 14:45:35+00:00,0,0.0,5.857,6.153,0.216926,0.227889
96,2020-01-26 16:26:28+00:00,-5,-4.0,9.099,8.651,0.216643,0.205976
97,2020-04-19 09:30:06+00:00,10,15.0,11.859,7.656,0.263533,0.170133
98,2020-04-23 01:37:01+00:00,3,5.0,8.366,7.847,0.157849,0.148057


In [ ]:
def wordList(tText):
    
    doc = nlp(tText)
    ### TOKENIZING ###
    ### REMOVING STOP WORDS ###
    wList = [token for token in doc if not token.is_stop]
    ### NORMALIZING ###
        ### STEMMING ###
        ### LEMMATIZATION ###
    wList = [token.lemma_ for token in wList]
    ### REMOVE PUNCTUATION ###
    punctuations="?:!.,;"
    for word in wList:
        if word in punctuations:
            wList.remove(word)
    ### POS filtering ###
    
    #wString = ""
    #for each in wList:
    #    wString += each + " "
    return wList